In [ ]:
import os
import cv2
import numpy as np
import kagglehub
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms

# ---- Dataset Parameters ----
img_width, img_height = 200, 200
classes = ["Fighting", "Robbery", "Shooting", "Stealing", "NormalVideos"]
num_classes = len(classes)

def balance_dataset(directory, classes, max_samples_per_class=7140):
    X, y = [], []
    class_to_label = {class_name: i for i, class_name in enumerate(classes)}

    for class_name in classes:
        class_dir = os.path.join(directory, class_name)
        if not os.path.exists(class_dir):
            continue

        image_files = os.listdir(class_dir)[:max_samples_per_class]

        for image_file in image_files:
            image_path = os.path.join(class_dir, image_file)
            image = cv2.imread(image_path)
            if image is not None:
                image = cv2.resize(image, (img_width, img_height))
                image = image / 255.0
                X.append(image)
                y.append(class_to_label[class_name])

    return np.array(X), np.array(y)

path = kagglehub.dataset_download("odins0n/ucf-crime-dataset")
print("Path to dataset files:", path)

train_dir = os.path.join(path, "Train")
test_dir = os.path.join(path, "Test")

X_train, y_train = balance_dataset(train_dir, classes)
X_test, y_test = balance_dataset(test_dir, classes)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=42)

class CrimeDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        image = self.X[idx]
        image = image.transpose((2, 0, 1))  
        image = torch.tensor(image, dtype=torch.float32)
        label = torch.tensor(self.y[idx], dtype=torch.long)
        if self.transform:
            image = self.transform(image)
        return image, label

train_dataset = CrimeDataset(X_train, y_train)
val_dataset = CrimeDataset(X_val, y_val)
test_dataset = CrimeDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad = False 

model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

def train_model(model, train_loader, val_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        train_loss, correct, total = 0, 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        acc = 100. * correct / total
        print(f"Epoch {epoch+1} | Loss: {train_loss:.3f} | Train Accuracy: {acc:.2f}%")


train_model(model, train_loader, val_loader)

def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

    acc = 100. * correct / total
    print(f"Test Accuracy: {acc:.2f}%")

evaluate(model, test_loader)


100%|██████████| 11.0G/11.0G [01:23<00:00, 141MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/odins0n/ucf-crime-dataset/versions/1


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 215MB/s]


Epoch 1 | Loss: 692.670 | Train Accuracy: 72.65%
Epoch 2 | Loss: 469.247 | Train Accuracy: 82.41%
Epoch 3 | Loss: 401.379 | Train Accuracy: 85.22%
Epoch 4 | Loss: 366.114 | Train Accuracy: 86.62%
Epoch 5 | Loss: 333.263 | Train Accuracy: 87.99%
Epoch 6 | Loss: 312.071 | Train Accuracy: 88.70%
Epoch 7 | Loss: 293.925 | Train Accuracy: 89.44%
Epoch 8 | Loss: 275.654 | Train Accuracy: 90.25%
Epoch 9 | Loss: 259.615 | Train Accuracy: 90.92%
Epoch 10 | Loss: 240.963 | Train Accuracy: 91.44%
Test Accuracy: 33.94%


In [ ]:
model_save_path = "resnet50_ucf_crime.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to: {model_save_path}")


In [ ]:
from google.colab import files
files.download('resnet50_ucf_crime.pth')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 20.4 MB/s eta 0:00:00
